In [1]:
!pip install scikit-surprise

In [1]:
from fastai.collab import *
from fastai.tabular import *

In [2]:
import pandas as pd

In [3]:
df_train = pd.read_csv('train.csv')
df_train.head()

,user_sequence,user_id,challenge_sequence,challenge
0,4576_1,4576,1,CI23714
1,4576_2,4576,2,CI23855
2,4576_3,4576,3,CI24917
3,4576_4,4576,4,CI23663
4,4576_5,4576,5,CI23933


In [4]:
df_train.shape

(903916, 4)

In [5]:
df_test = pd.read_csv('test.csv')
df_test.head()

,user_sequence,user_id,challenge_sequence,challenge
0,4577_1,4577,1,CI23855
1,4577_2,4577,2,CI23933
2,4577_3,4577,3,CI24917
3,4577_4,4577,4,CI24915
4,4577_5,4577,5,CI23714


In [6]:
df_test.shape

(397320, 4)

In [7]:
train = df_train.append(df_test)
train.shape

(1301236, 4)

In [8]:
cols = ['user_sequence','challenge_sequence']
train.drop(cols,axis=1,inplace=True)
train['rating']=1
train.head()

,user_id,challenge,rating
0,4576,CI23714,1
1,4576,CI23855,1
2,4576,CI24917,1
3,4576,CI23663,1
4,4576,CI23933,1


In [9]:
from surprise import Reader, Dataset

In [10]:
ratings_dict = {'itemID': list(train.challenge),
                'userID': list(train.user_id),
                'rating': list(train.rating)}
df = pd.DataFrame(ratings_dict)

In [11]:
reader = Reader(rating_scale=(-0.1, 1.0))

In [12]:
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

In [14]:
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import NMF

# svd
algo_1 = SVD()
cross_validate(algo_1, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0226  0.0226  0.0227  0.0225  0.0227  0.0226  0.0001  
MAE (testset)     0.0078  0.0078  0.0078  0.0078  0.0078  0.0078  0.0000  
Fit time          69.38   62.42   63.00   63.45   73.46   66.34   4.36    
Test time         2.59    2.33    2.31    2.70    2.55    2.50    0.15    


{'test_rmse': array([0.022581, 0.022572, 0.022667, 0.022457, 0.022661]),
 'test_mae': array([0.007815, 0.007793, 0.007836, 0.007802, 0.007832]),
 'fit_time': (69.3848488330841,
  62.41615414619446,
  62.995245695114136,
  63.44920063018799,
  73.46149206161499),
 'test_time': (2.589520215988159,
  2.329153060913086,
  2.311291217803955,
  2.701282024383545,
  2.551966428756714)}

In [16]:
# nmf
algo_2 = NMF()
cross_validate(algo_2, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0612  0.0612  0.0615  0.0614  0.0614  0.0613  0.0001  
MAE (testset)     0.0611  0.0611  0.0613  0.0612  0.0612  0.0612  0.0001  
Fit time          81.42   82.40   83.12   81.50   74.53   80.59   3.09    
Test time         2.44    2.38    2.39    2.31    2.86    2.47    0.20    


{'test_rmse': array([0.061202, 0.061238, 0.061451, 0.061358, 0.061357]),
 'test_mae': array([0.061052, 0.061082, 0.061287, 0.061208, 0.061192]),
 'fit_time': (81.42433428764343,
  82.39827370643616,
  83.11579728126526,
  81.49846768379211,
  74.53083348274231),
 'test_time': (2.436321496963501,
  2.377495527267456,
  2.386823892593384,
  2.308434009552002,
  2.859050989151001)}

In [ ]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.25)
predictions = algo_1.fit(trainset).test(testset)

In [20]:
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 0.0233


0.023311471117247105

In [ ]:
trainset = data.build_full_trainset()

In [26]:
testset = trainset.build_anti_testset()
predictions = algo_1.test(testset)

MemoryError: 